In [ ]:
!pip install flask
!pip install flask-ngrok
!pip install flask flask-ngrok
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Set your ngrok authentication token
ngrok.set_auth_token("2ryNcDZvZlVMDlan1uTjRQIu7wF_6m13bULwVxUAPYQTgsdny")


In [ ]:
from flask import Flask, render_template_string, jsonify, request
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Updated dummy data for bed availability with categories
beds = [
    # ICU Beds
    {"id": 1, "bed_number": "ICU 1", "status": "Available", "category": "ICU"},
    {"id": 2, "bed_number": "ICU 2", "status": "Occupied", "category": "ICU"},
    {"id": 3, "bed_number": "ICU 3", "status": "Available", "category": "ICU"},
    {"id": 4, "bed_number": "ICU 4", "status": "Occupied", "category": "ICU"},

    # Emergency Beds
    {"id": 5, "bed_number": "Emergency 1", "status": "Available", "category": "Emergency"},
    {"id": 6, "bed_number": "Emergency 2", "status": "Occupied", "category": "Emergency"},
    {"id": 7, "bed_number": "Emergency 3", "status": "Available", "category": "Emergency"},
    {"id": 8, "bed_number": "Emergency 4", "status": "Occupied", "category": "Emergency"},

    # General Beds
    {"id": 9, "bed_number": "General 1", "status": "Available", "category": "General"},
    {"id": 10, "bed_number": "General 2", "status": "Occupied", "category": "General"},
    {"id": 11, "bed_number": "General 3", "status": "Available", "category": "General"},
    {"id": 12, "bed_number": "General 4", "status": "Occupied", "category": "General"},
    {"id": 13, "bed_number": "General 5", "status": "Available", "category": "General"},
    {"id": 14, "bed_number": "General 6", "status": "Occupied", "category": "General"},
    {"id": 15, "bed_number": "General 7", "status": "Available", "category": "General"},
    {"id": 16, "bed_number": "General 8", "status": "Occupied", "category": "General"}
]

# Route to display beds
@app.route('/')
def index():
    # Calculate statistics
    total_beds = len(beds)
    available_beds = sum(1 for bed in beds if bed["status"] == "Available")
    occupied_beds = total_beds - available_beds

    # HTML with improved UI
    html = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Hospital Bed Availability Dashboard</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f4f4f9;
            }
            h1 {
                text-align: center;
                color: #333;
            }
            .stats {
                text-align: center;
                margin-bottom: 20px;
            }
            .stats div {
                display: inline-block;
                margin: 10px;
                padding: 15px;
                background-color: #e3f2fd;
                border: 1px solid #2196f3;
                border-radius: 5px;
            }
            .search-bar {
                text-align: center;
                margin-bottom: 20px;
            }
            .search-bar input {
                padding: 10px;
                width: 50%;
                border: 1px solid #ccc;
                border-radius: 5px;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }
            table, th, td {
                border: 1px solid #ddd;
            }
            th, td {
                padding: 10px;
                text-align: center;
            }
            th {
                background-color: #2196f3;
                color: white;
            }
            .ICU {
                background-color: #f8bbd0;
            }
            .Emergency {
                background-color: #ffcc80;
            }
            .General {
                background-color: #c5e1a5;
            }
            button {
                padding: 5px 10px;
                cursor: pointer;
                background-color: #2196f3;
                color: white;
                border: none;
                border-radius: 3px;
            }
            button:hover {
                background-color: #1976d2;
            }
        </style>
    </head>
    <body>
        <h1>Hospital Bed Availability Dashboard</h1>
        <div class="stats">
            <div>Total Beds: {{ total_beds }}</div>
            <div>Available Beds: {{ available_beds }}</div>
            <div>Occupied Beds: {{ occupied_beds }}</div>
        </div>
        <div class="search-bar">
            <input type="text" id="searchInput" onkeyup="filterTable()" placeholder="Search for beds or categories...">
        </div>
        <table id="bedsTable">
            <thead>
                <tr>
                    <th>Bed Number</th>
                    <th>Category</th>
                    <th>Status</th>
                    <th>Action</th>
                </tr>
            </thead>
            <tbody>
                {% for bed in beds %}
                <tr class="{{ bed['category'] }}">
                    <td>{{ bed['bed_number'] }}</td>
                    <td>{{ bed['category'] }}</td>
                    <td>{{ bed['status'] }}</td>
                    <td>
                        <button onclick="updateStatus({{ bed['id'] }}, '{{ 'Available' if bed['status'] == 'Occupied' else 'Occupied' }}')">
                            {{ 'Mark as Available' if bed['status'] == 'Occupied' else 'Mark as Occupied' }}
                        </button>
                    </td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        <script src="https://cdn.jsdelivr.net/npm/axios/dist/axios.min.js"></script>
        <script>
            function updateStatus(bedId, newStatus) {
                axios.post('/update', { id: bedId, status: newStatus })
                    .then(response => {
                        if (response.data.success) {
                            location.reload();
                        }
                    });
            }
            function filterTable() {
                let input = document.getElementById("searchInput").value.toLowerCase();
                let rows = document.getElementById("bedsTable").getElementsByTagName("tr");
                for (let i = 1; i < rows.length; i++) {
                    let cells = rows[i].getElementsByTagName("td");
                    let match = false;
                    for (let cell of cells) {
                        if (cell.textContent.toLowerCase().includes(input)) {
                            match = true;
                            break;
                        }
                    }
                    rows[i].style.display = match ? "" : "none";
                }
            }
        </script>
    </body>
    </html>
    """
    return render_template_string(html, beds=beds, total_beds=total_beds, available_beds=available_beds, occupied_beds=occupied_beds)

# Route to update bed status
@app.route('/update', methods=['POST'])
def update():
    global beds
    data = request.json
    bed_id = data['id']
    new_status = data['status']
    for bed in beds:
        if bed['id'] == bed_id:
            bed['status'] = new_status
            break
    return jsonify({'success': True})

# Start the Flask app and ngrok tunnel
def run_app():
    public_url = ngrok.connect(5000)  # Expose Flask app on port 5000
    print(f"Public URL: {public_url}")
    app.run()

run_app()


Public URL: NgrokTunnel: "https://098f-35-229-239-76.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
